# Temporary Assistance for Needy Families (TANF)

Currently in development, TANF is a state-level benefit program.

From benefits.gov:

> The Temporary Assistance for Needy Families (TANF) program provides grant funds to states and territories to provide families with financial assistance and related support services. State-administered programs may include childcare assistance, job preparation, and work assistance.

## General formula

To calculate TANF entitlement, we use the general computation tree below (may vary depending on state and county):

- `tanf`: TANF entitlement
  - `tanf_amount_if_eligible`: amount if eligible
    - Definition: `tanf_max_amount - tanf_countable_income`
    - `tanf_max_amount`: maximum amount
      - **Parameters**: the maximum amount defined by different state TANF programs may vary due to these factors:
        - Household size: How many people live in the household.
        - "Region": Some states may define a "region"-level distinction for maximum amounts; for example, California (CalWORKS) defines two regions based on what county the household resides in.
        - Individual household properties such as caregiver or disability status.
        - Example: California (CalWORKS) has different maximum amounts based on these factors, which they define as "exempt/non-exempt".
        - State-specific policies.
    - `tanf_countable_income`: Amount deducted from TANF maximum amount based on income.
      - `tanf_gross_earned_income`: earned income.
        - **Parameter**: list of earned income sources summed.
      - `tanf_gross_unearned_income`: unearned income. (ADD DEF HERE)
        - **Parameter**: list of unearned income sources summed.
      - `deductions`: deductions from assessed income.
        - `earnings_deduction`: deduction amount based on earnings.
          - **Parameter**: percentage of earnings deducted from gross earned income.
          - **Parameter**: flat amount deducted from the household's gross earned income.
          - **Parameter**: flat amount deducted from each earner's gross earned income.
  - `is_tanf_eligible`: whether eligible for TANF
    - `is_tanf_enrolled`: whether a family is already enrolled in TANF.
    - `is_tanf_demographically_eligible`: demographic definition of TANF eligibility, which is mostly constant across the US.
      - Definition: If there are children (ages 0-17) present in the household, pregnant people, or there are people aged 18 years old that are currently enrolled in a school, the family is demographically eligible for TANF.
    - `is_tanf_economically_eligible`: Whether the family has sufficiently low income to qualify for eligibility.
      - `is_tanf_enrolled`: whether a family is already enrolled in TANF.
      - `is_tanf_continuous_eligible`:
        - `tanf_eligibility_income`: income measure used to assess eligibility for TANF.
          - **Parameters**: income definition varies depending on state policies:
            - **Parameter**: deductions from income per earner
            - **Parameter**: deductions from income per household
        - `tanf_max_amount` (defined above)
      - `is_tanf_initial_eligible`:
        - This variable is very similar to `is_continuous_eligible`, except for the initial employment deductions that are applied to determininig initial eligibility.
        - `tanf_eligibility_income`: income measure used to assess eligibility for TANF.
          - **Parameters**: income definition varies depending on state policies:
            - **Parameter**: deductions from income per earner
            - **Parameter**: deductions from income per household
        - `tanf_max_amount` (defined above)

In [1]:
from openfisca_us import IndividualSim
import pandas as pd
import plotly.express as px

sim_emp = IndividualSim(year=2022)
sim_emp.add_person(name="adult", age=30, employment_income=250 * 12)
sim_emp.add_person(name="child", age=10)
sim_emp.add_spm_unit(name="spm_unit", members=["adult", "child"])
sim_emp.add_household(name="household", members=["adult", "child"], state_code="IL")

print("TANF: ", sim_emp.calc("tanf") / 12)

What if their earnings change?
They receive the maximum allotment of \$250 per month until they reach \$720 in monthly earnings, plus the \$95 emergency allotment, coming to \$345.
When their income hits \$980 per month, their normal allotment falls to \$155, at which point their total allotment stabilizes at \$250.
The normal allotment then continues to fall until their earnings reach \$1,350, at which point their continue to receive the \$20 minimum monthly allotment until they reach \$2,150 per month, 200% of the poverty line.
At 200% of the poverty line, both their normal and full allotment fall to \$0.

In [ ]:
LABELS = dict(
    employment_income="Monthly employment income",
    dividend_income="Monthly dividend income",
    income="Monthly income",
    income_source="Income source",
    tanf="Monthly TANF allotment",
    mtr="Marginal tax rate from TANF?",
    allotment="TANF allotment",
)

def make_df(state_code, enrolled):
    sim = IndividualSim(year=2022)
    sim.add_person(name="adult", age=30, employment_income=250 * 12)
    sim.add_person(name="child", age=10)
    sim.add_spm_unit(name="spm_unit", members=["adult", "child"], is_tanf_enrolled=enrolled)
    sim.add_household(name="household", members=["adult", "child"], state_code=state_code)

    sim.vary("employment_income", max=1000 * 12, step=120)

    df = pd.DataFrame(
        dict(
            employment_income=sim.calc("employment_income")[0],
            enrolled="Enrolled" if enrolled else "Not Enrolled",
            state_code = state_code,
            tanf=sim.calc("tanf")[0],
            mtr=-sim_emp.deriv("tanf", "employment_income"),
        )
    )

    df[["employment_income", "tanf"]] = (
        df[["employment_income", "tanf"]] / 12
    ).round()

    return df

fig = px.line(
    make_df("IL", True),
    "employment_income",
    "tanf",
    labels=LABELS,
    title="TANF allotment for a two-person household in Illinois",
)

fig.update_layout(xaxis_tickformat="$,", yaxis_tickformat="$,")
fig.show()

TypeError: len() of unsized object

In [ ]:
emp_df_combined = pd.concat([make_df("IL", True), make_df("IL", False), make_df("CA", True), make_df("CA", False)])

fig = px.line(
    emp_df_combined,
    "employment_income",
    "tanf",
    color="enrolled",
    labels=LABELS,
    animation_frame="state_code",
    title="TANF allotment for a two-person household in Illinois",
)

fig.update_layout(xaxis_tickformat="$,", yaxis_tickformat="$,", yaxis_range=[0,emp_df_combined.tanf.max()*1.1])
fig.show()

In [ ]:
emp_df_combined

,employment_income,enrolled,state_code,tanf
0,0.0,Enrolled,IL,435.0
1,10.0,Enrolled,IL,432.0
2,20.0,Enrolled,IL,430.0
3,30.0,Enrolled,IL,428.0
4,40.0,Enrolled,IL,425.0
...,...,...,...,...
96,960.0,Not Enrolled,CA,0.0
97,970.0,Not Enrolled,CA,0.0
98,980.0,Not Enrolled,CA,0.0
99,990.0,Not Enrolled,CA,0.0


We can also view their marginal tax rate from the program, revealing that SNAP phases out at 36 cents on the dollar through the phase-out region.
The emergency allotment shrinks this region.
The person also experiences a cliff (infinite marginal tax rate) when they hit 200% of the poverty line.

In [ ]:
fig = px.line(
    emp_df,
    "employment_income",
    "mtr",
    color="allotment",
    labels=LABELS,
    title="SNAP marginal tax rate for a one-person household in California with $600 monthly housing costs",
)
fig.update_layout(
    xaxis_tickformat="$,", yaxis_tickformat=".0%", yaxis_range=[0, 1]
)
fig.show()

NameError: name 'emp_df' is not defined

### How housing costs affect a four-person household's SNAP allotments

We can also visualize how other household characteristics affect SNAP allotments.
For example, what if we vary the housing costs of a four-person household with \$2,000 monthly income?

Their normal SNAP allotment is \$409 per month if their housing costs are less than \$720 per month. If they spend \$1,310 per month on housing, their normal allotment rises 44% to \$588 per month.

However, because their maximum allotment exceeds their normal allotment by at least \$95 regardless of their housing costs, they will receive the maximum allotment regardless of housing costs as a result of the emergency allotment.

In [ ]:
sim_rent = IndividualSim(year=2022)
sim_rent.add_person(name="parent1", employment_income=1000 * 12)
sim_rent.add_person(name="parent2", employment_income=1000 * 12)
sim_rent.add_person(name="child1")
sim_rent.add_person(name="child2")
sim_rent.add_spm_unit(
    name="spm_unit",
    members=["parent1", "parent2", "child1", "child2"],
)

sim_rent.vary("housing_cost", max=2000 * 12, step=120)

rent_df_full = pd.DataFrame(
    dict(
        allotment="Full",
        housing_cost=sim_rent.calc("housing_cost")[0],
        snap=sim_rent.calc("snap")[0],
        housing_subsidy_rate=sim_rent.deriv("snap", "housing_cost"),
    )
)
rent_df_normal = pd.DataFrame(
    dict(
        allotment="Normal",
        housing_cost=sim_rent.calc("housing_cost")[0],
        snap=sim_rent.calc("snap_normal_allotment")[0],
        housing_subsidy_rate=sim_rent.deriv(
            "snap_normal_allotment", "housing_cost"
        ),
    )
)
rent_df = pd.concat([rent_df_full, rent_df_normal])
rent_df[["housing_cost", "snap"]] = (
    rent_df[["housing_cost", "snap"]] / 12
).round()

fig = px.line(
    rent_df,
    "housing_cost",
    "snap",
    color="allotment",
    labels=LABELS,
    title="SNAP allotment for a four-person household in California with $1,000 monthly earnings",
)
fig.update_layout(
    xaxis_tickformat="$,",
    yaxis_tickformat="$,",
    yaxis_range=[0, rent_df.snap.max() * 1.1],
)
fig.show()

As before, we can also see the rate at which SNAP increases with housing costs.
For this household, the normal SNAP allotment subsidizes housing costs at a rate of 30% for housing costs ranging from \$720 to \$1,310 per month.
The emergency allotment nullifies this effect.

In [ ]:
fig = px.line(
    rent_df,
    "housing_cost",
    "housing_subsidy_rate",
    color="allotment",
    labels=LABELS,
    title="SNAP housing subsidy rate for a one-person household in California with $1,000 monthly earnings",
)
fig.update_layout(xaxis_tickformat="$,", yaxis_tickformat=".0%")
fig.show()

### How unearned income compares to earned income

SNAP exempts 20% of earned income from net income calculations.
This earned income deduction does not apply to unearned income, such as Social Security, unemployment allotments, interest, and dividends.

To illustrate the effect of this provision, let's return to the first example of a single person with \$1,000 monthly income and \$600 monthly rent.
When their income was earned, they received a normal allotment of \$145, plus a \$105 emergency allotment to bring them to the one-person maximum of \$250.
If their income is unearned, for example if it's from dividends, their normal allotment falls to \$55, though the emergency allotment fills the gap, bringing their total allotment again to \$250.

In [ ]:
sim_div = IndividualSim(year=2022)
sim_div.add_person(name="person", dividend_income=1000 * 12)
sim_div.add_spm_unit(
    name="spm_unit", members=["person"], housing_cost=600 * 12
)

print(
    "SNAP normal allotment: ",
    round(sim_div.calc("snap_normal_allotment")[0] / 12),
)
print(
    "SNAP emergency allotment: ",
    round(sim_div.calc("snap_emergency_allotment")[0] / 12),
)
print("Total SNAP: ", round(sim_div.calc("snap")[0] / 12))

As a result of the earned income deduction, this household will have a larger SNAP benefit if their income comes from employment, if their monthly income is between \$570 and \$980.

In [ ]:
sim_div.vary("dividend_income", max=2500 * 12, step=120)

div_df_full = pd.DataFrame(
    dict(
        dividend_income=sim_div.calc("dividend_income")[0],
        allotment="Full",
        snap=sim_div.calc("snap")[0],
        mtr=-sim_div.deriv("snap", "dividend_income"),
    )
)
div_df_normal = pd.DataFrame(
    dict(
        dividend_income=sim_div.calc("dividend_income")[0],
        allotment="Normal",
        snap=sim_div.calc("snap_normal_allotment")[0],
        mtr=-sim_div.deriv("snap_normal_allotment", "dividend_income"),
    )
)
div_df = pd.concat([div_df_full, div_df_normal])
div_df[["dividend_income", "snap"]] = (
    div_df[["dividend_income", "snap"]] / 12
).round()

# Combine employment and dividend income DataFrames.
emp_df["income_source"] = "Earned"
div_df["income_source"] = "Unearned"
emp_div_df = pd.concat(
    [
        emp_df.rename(columns=dict(employment_income="income")),
        div_df.rename(columns=dict(dividend_income="income")),
    ]
)

fig = px.line(
    emp_div_df[emp_div_df.allotment == "Full"],
    "income",
    "snap",
    color="income_source",
    labels=LABELS,
    title="Full SNAP allotment for a one-person household in California with $600 monthly housing costs",
)
fig.update_layout(xaxis_tickformat="$,", yaxis_tickformat="$,")
fig.show()

While the SNAP marginal tax rate is 36% for employment income between \$710 and \$960, it is 45% for dividend income between \$570 and \$760.

In [ ]:
fig = px.line(
    emp_div_df[emp_div_df.allotment == "Full"],
    "income",
    "mtr",
    color="income_source",
    labels=LABELS,
    title="Full SNAP marginal tax rate for a one-person household in California with $600 monthly housing costs",
)
fig.update_layout(
    xaxis_tickformat="$,", yaxis_tickformat=".0%", yaxis_range=[0, 1]
)
fig.show()

In the absence of the emergency allotment, SNAP allotments differ between earned and unearned income over larger income ranges.

In [ ]:
fig = px.line(
    emp_div_df[emp_div_df.allotment == "Normal"],
    "income",
    "snap",
    color="income_source",
    labels=LABELS,
    title="Normal SNAP allotment for a one-person household in California with $600 monthly housing costs",
)
fig.update_layout(xaxis_tickformat="$,", yaxis_tickformat="$,")
fig.show()

Without emergency allotments, the marginal tax rates remain at 36% and 45% for earned and unearned income, respectively, over larger income ranges.

In [ ]:
fig = px.line(
    emp_div_df[emp_div_df.allotment == "Normal"],
    "income",
    "mtr",
    color="income_source",
    labels=LABELS,
    title="Normal SNAP marginal tax rate for a one-person household in California with $600 monthly housing costs",
)
fig.update_layout(
    xaxis_tickformat="$,", yaxis_tickformat=".0%", yaxis_range=[0, 1]
)
fig.show()